In [1]:
from pyspark import SparkConf, SparkContext
conf=SparkConf().setMaster("local").setAppName("key-value_rdd_op_joins")
sc=SparkContext(conf=conf)

# Operations
-'groupByKey'

    -'KeyValueRDD.groupByKey()'  
    -그룹핑 후에 특정 Transformations 같은 연산
    -key 값이 있는 상태에서 시작
      
      
-'groupBy()'

    -'RDD.groupBy(numPatitions=None,patitionFunc=<function portable_hash>)'
    -함수에 의해서 그룹이 생기는 연산

In [5]:
rdd=sc.parallelize([
    ("짜장면",15),
    ("짬뽕",10),
    ("짜장면",5)
])

g_rdd=rdd.groupByKey()
g_rdd.collect()

[('짜장면', <pyspark.resultiterable.ResultIterable at 0x25f262baf70>),
 ('짬뽕', <pyspark.resultiterable.ResultIterable at 0x25f2648a4f0>)]

In [7]:
g_rdd.mapValues(len).collect()

[('짜장면', 2), ('짬뽕', 1)]

In [8]:
g_rdd.mapValues(list).collect()

[('짜장면', [15, 5]), ('짬뽕', [10])]

In [10]:
#groupBy는 그룹핑할 키에 대한 정의를 개발자가 직접해줘야한다.
grouped = sc.parallelize([
    "C","C#","JAVA","C++","JAVASCRIPT","PYTHON"
    
]).groupBy(lambda x : x[0]).collect()

grouped

[('C', <pyspark.resultiterable.ResultIterable at 0x25f264c8340>),
 ('J', <pyspark.resultiterable.ResultIterable at 0x25f262b85b0>),
 ('P', <pyspark.resultiterable.ResultIterable at 0x25f2621fee0>)]

In [11]:
for k,v in grouped:
    print(k, list(v))

C ['C', 'C#', 'C++']
J ['JAVA', 'JAVASCRIPT']
P ['PYTHON']


In [13]:
#groupBykey는 K-V RDD를 사용할 때 key가 알아서 그룹핑의 기준이 도니다.

x=sc.parallelize([
 ("math",7),("math",2),("english",7),("english",3),("english",4),("science",7),("science",6),("science",9),("science",5)   
    
],3)

y=x.groupByKey()

In [16]:
print(y.getNumPartitions())

3


In [18]:
#파티션의 개수를 바꿀수있다.
y=x.groupByKey(2)
y.getNumPartitions()

2

In [20]:
y.collect()

[('math', <pyspark.resultiterable.ResultIterable at 0x25f2636ac40>),
 ('science', <pyspark.resultiterable.ResultIterable at 0x25f262de6d0>),
 ('english', <pyspark.resultiterable.ResultIterable at 0x25f262de190>)]

In [21]:
for t in y.collect():
    print(t[0],list(t[1])) # t[0]: key,t[1]: 그룹핑에 의해 묶인 값

math [7, 2]
science [7, 6, 9, 5]
english [7, 3, 4]


# reduceByKey
-'KeyValueRDD,reduceByKey(<func>,numPartitions=None,partitionFunc=<function portable_hash>)
-주어지는 key를 기준으로 Group을 만들고 합쳐준다.
-Transformations 함수이다.

In [23]:
from operator import add

rdd=sc.parallelize([
    ("짜장면",15),
    ("짬뽕",10),
    ("짜장면",5)
])

rdd.reduceByKey(add).collect()

[('짜장면', 20), ('짬뽕', 10)]

개념적으로  groupByKey + reduce입니다. groupByKey보다 reduceByKey를 쓰는게 훨씬 빠르다

In [25]:
x=sc.parallelize([
 ("math",7),("math",2),("english",7),("english",3),("english",4),("science",7),("science",6),("science",9),("science",5)   
    
],3)

x.reduceByKey(lambda a,b : a+b).collect()

[('english', 14), ('math', 9), ('science', 27)]

# mapValues

 
 -'KeyValueRDD.mapValues(<func>)'
    
 -함수를 'Value'에만 적용합니다.
    
    -파티션과 키는 그 위치에 그대로 있다    
    
 -'Transformations 작업이다.

In [ ]:
rdd=sc.para

In [28]:
rdd=sc.parallelize([
("하의",["청바지","반바지","치마"]),
("상의",["니트","반팔","긴팔","나시"])
])

rdd.mapValues(lambda x : len(x)).collect()

[('하의', 3), ('상의', 4)]

key가 아닌 value에만 적용할 함수를 만들 수가 있기 때문에 데이터의 파티션이 변경될 염려가 없다.

# countByKey
-action이다.

-각 키가 가진 요소들의 개수를 센다. 

-KeyValueRDD.countByKey(<func>)

In [30]:
rdd=sc.parallelize([
("하의",["청바지","반바지","치마"]),
("상의",["니트","반팔","긴팔","나시"])
])

rdd.countByKey()


defaultdict(int, {'하의': 1, '상의': 1})

# keys()
-모든 key를 가진 RDD를 생성한다.

-'keys()'는 파티션을 유지하거나 키가 굉장히 많은 경우가 있기 때문에 Transformation이다.



In [31]:
rdd.keys()

PythonRDD[67] at RDD at PythonRDD.scala:53

In [32]:
rdd.keys().collect()

['하의', '상의']

In [35]:
x=sc.parallelize([
 ("math",7),("math",2),("english",7),("english",3),("english",4),("science",7),("science",6),("science",9),("science",5)   
    
],3)

print(x.keys().count()) #키의 총 개수
print(x.keys().distinct().count()) #키의 종류 개수

9
3


#  Joins

In [37]:
#inner join : 서로간에 존재하는 키만 합쳐준다.
rdd1=sc.parallelize([
    ("foo",1),
    ("goo",2),
    ("hoo",3)  
])

rdd2=sc.parallelize([
    ("foo",1),
    ("goo",2),
    ("goo",10),
    ("moo",5)
])

rdd1.join(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10))]

# OuterJoin
-기준이 되는 한 쪽에는 데이터가 있고, 다른쪽에는 데이터가 없는 경우

    - 설정한 기준에 따라서 기준에 맞는 데이터가 항상 남아있는다.

- leftOuterJoin : 왼쪽에 있는 rdd가 기준이 된다. (함수를 호출하는 쪽)
- rightOuterJoin : 오른쪽에는 있는 rdd가 기준이 된다 (함수를 매개변수로 들어가는 쪽)

In [38]:
rdd1.leftOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('goo', (2, 2)), ('goo', (2, 10)), ('hoo', (3, None))]

In [39]:
rdd1.rightOuterJoin(rdd2).collect()

[('foo', (1, 1)), ('moo', (None, 5)), ('goo', (2, 2)), ('goo', (2, 10))]